In [ ]:
import os
import json
import boto3
import time
import sagemaker
from sagemaker.model import Model
from sagemaker.model_metrics import MetricsSource, ModelMetrics
from sagemaker.workflow.experiments import Experiment, Trial


In [ ]:
# -----------------------------
# Configuration
# -----------------------------
BUCKET = "mlops-creditcard"
EXPERIMENT_NAME = "creditcard-fraud"
MODEL_PACKAGE_GROUP = "creditcard-fraud-models"

LOCAL_MODEL_PATH = "./CREDITCARD/MODEL/model.pkl"
LOCAL_METRICS_PATH = "./CREDITCARD/MODEL/metrics.json"

ROLE = sagemaker.get_execution_role()
REGION = boto3.Session().region_name
sm_client = boto3.client("sagemaker")
s3_client = boto3.client("s3")


In [ ]:
timestamp = time.strftime("%Y%m%d-%H%M%S")
trial_name = f"run-{timestamp}"

# Create experiment if not exists
try:
    experiment = Experiment.load(experiment_name=EXPERIMENT_NAME)
except Exception:
    experiment = Experiment.create(
        experiment_name=EXPERIMENT_NAME,
        description="Credit Card Fraud Detection Experiment"
    )

# Create trial
trial = Trial.create(
    trial_name=trial_name,
    experiment_name=EXPERIMENT_NAME
)


In [ ]:
s3_base_path = f"experiments/{EXPERIMENT_NAME}/{trial_name}"

model_s3_uri = f"s3://{BUCKET}/{s3_base_path}/model/model.pkl"
metrics_s3_uri = f"s3://{BUCKET}/{s3_base_path}/metrics/metrics.json"

s3_client.upload_file(
    LOCAL_MODEL_PATH, BUCKET, f"{s3_base_path}/model/model.pkl"
)

s3_client.upload_file(
    LOCAL_METRICS_PATH, BUCKET, f"{s3_base_path}/metrics/metrics.json"
)


In [ ]:
with open(LOCAL_METRICS_PATH) as f:
    metrics = json.load(f)

for k, v in metrics.items():
    trial.add_metric(name=k, value=float(v))


In [ ]:
image_uri = sagemaker.image_uris.retrieve(
    framework="sklearn",
    region=REGION,
    version="1.2-1",
    py_version="py3",
    instance_type="ml.m5.large"
)

model = Model(
    image_uri=image_uri,
    model_data=model_s3_uri,
    role=ROLE,
    sagemaker_session=sagemaker.Session()
)


In [ ]:
model_metrics = ModelMetrics(
    model_statistics=MetricsSource(
        s3_uri=metrics_s3_uri,
        content_type="application/json"
    )
)


In [ ]:
model_package = model.register(
    content_types=["application/json"],
    response_types=["application/json"],
    inference_instances=["ml.m5.large"],
    transform_instances=["ml.m5.large"],
    model_package_group_name=MODEL_PACKAGE_GROUP,
    approval_status="PendingManualApproval",
    model_metrics=model_metrics,
    description="RandomForest credit card fraud model"
)


In [ ]:
model_package_arn = model_package.model_package_arn

sm_client.add_tags(
    ResourceArn=model_package_arn,
    Tags=[
        {"Key": "role", "Value": "challenger"},
        {"Key": "status", "Value": "staging"},
        {"Key": "experiment", "Value": EXPERIMENT_NAME},
        {"Key": "trial", "Value": trial_name}
    ]
)
